In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. 데이터 가져오기

In [33]:
t_bat16=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2016.csv",encoding='utf-8',engine='python')
t_bat17=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2017.csv",encoding='utf-8',engine='python')
t_bat18=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2018.csv",encoding='utf-8',engine='python')
t_bat19=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2019.csv",encoding='utf-8',engine='python')

In [34]:
#모든 열 다 출력하기
pd.options.display.max_columns=None

## 2. 16,17,18,19년도 팀,상대팀 별 타율 만들기

In [35]:
team=['LG', 'HH', 'NC', 'HT', 'SK', 'KT', 'WO', 'LT', 'SS', 'OB']
team_vs=[t + vs for t in team for vs in team if t!=vs]

In [82]:
t_vs16=t_bat16.groupby(['T_ID','VS_T_ID'])
t_vs17=t_bat17.groupby(['T_ID','VS_T_ID'])
t_vs18=t_bat18.groupby(['T_ID','VS_T_ID'])
t_vs19=t_bat19.groupby(['T_ID','VS_T_ID'])

In [83]:
#data 분할 (하나의 리스트에 넣기)
def divi(t_vs):
    a=[]
    for i in range(len(team_vs)):
        a.append(list(t_vs)[i][1])   
    return a

In [84]:
data16=divi(t_vs16)
data17=divi(t_vs17)
data18=divi(t_vs18)
data19=divi(t_vs19)

In [93]:
data_list=[data16,data17,data18,data19]
train=[]  #9개별로 팀,상대팀이 다름
train_ans=[]

for k in range(90):
    for i in range(len(data_list)):
        for j in range(len(data_list)-1):
            if i!=j:
                train.append(pd.concat([data_list[i][k],data_list[j][k][:-3]])) #뒤에 3개 뺌
                train_ans.append(data_list[j][k][-3:])

In [96]:
test=[]
test_ans=[]
for k in range(90):
    for i in range(len(data_list)-1):
        test.append(pd.concat([data_list[i][k],data_list[3][k][:-3]])) #뒤에 3개 뺌
        test_ans.append(data_list[3][k][-3:])

In [ ]:
tr=[]
for i in range(90):
    j=0
    while j<9:
        j+=1
        train[i]['HIT_RATE']=train[i]['HIT']/train[i]['AB']  #타율 변수 생성
        tr.append(train[i][['T_ID','VS_T_ID','HIT_RATE']])   #팀,상대팀 별 타율만 남김
        num=list(range(1,int((len(train[i])/10)+1)))*10     #팀별 경기순서 추가
        tr=tr.assign(order=num)
        tr=tr.pivot(index='T_ID',columns='order',values='HIT_RATE') #팀당 경기별 타율 데이터셋 만들기
    
##############################################################################
##포기###########
##############################################################################

In [ ]:
#정답 셋과 train 셋 만드는 함수
t_bat=[t_bat16,t_bat17,t_bat18,t_bat19]
def make_answer_train(year):
    a=t_bat[year-2016][['G_ID','T_ID','AB','HIT']]
    
    #각 팀별로 마지막 20경기씩 뽑아서 밑으로 이어붙이기
    ans=pd.DataFrame()
    train=pd.DataFrame()
    
    for t in team:
        ans=pd.concat([ans,a[a['T_ID']==t][-20:]])
        train=pd.concat([train,a[a['T_ID']==t][:-20]])
        
    #팀별 AB,HIT합 구하기
    ans=ans.groupby(['T_ID']).sum()
    ans['HIT_RATE']=ans['HIT']/ans['AB'] #타율 계산
    ans=ans.drop(ans.columns[[0,1]],axis='columns')
    
    return ans,train

In [ ]:
def make_data(train,answer):
    train['HIT_RATE']=train['HIT']/train['AB']
    tr=train[['T_ID','HIT_RATE']]  #경기당 팀과 타율만 남김
    num=list(range(1,int((len(train)/10)+1)))*10  #팀별 경기순서 추가
    tr=tr.assign(order=num)
    tr=tr.pivot(index='T_ID',columns='order',values='HIT_RATE') #팀당 경기별 타율 데이터셋 만들기
    
    data=pd.merge(tr,answer,on='T_ID') #팀별 앞124경기와 후20경기 결과 합치기
    return data

## 3. 두개년도 합치고, 뒤 3경기 총타율 붙이기

## 6. 학습시킬 데이터 셋 만들기

In [ ]:
#팀, 상대팀, 월 을 기분으로 데이터 합치기
a=pd.merge(t16,t17,on=['T_ID','VS_T_ID','MONTH'])
b=pd.merge(a,t18,on=['T_ID','VS_T_ID','MONTH'])
data=pd.merge(b,t19,on=['T_ID','VS_T_ID','MONTH'])

a=pd.merge(data_cum,t16,on=['T_ID','VS_T_ID','MONTH'])
b=pd.merge(a,t17,on=['T_ID','VS_T_ID','MONTH'])
c=pd.merge(b,t18,on=['T_ID','VS_T_ID','MONTH'])
data=pd.merge(c,t19,on=['T_ID','VS_T_ID','MONTH'])

In [ ]:
data=data.reset_index(level=['MONTH']) #MONTH열을 index에서 값으로 변환
data=data.astype({'MONTH':'int'}) #MONTH 값 형식을 숫자로 변환

In [ ]:
data

16년, 17년, 18년 팀별 성적 데이터셋 이용

## 7. 모델학습

In [ ]:
# 출력 할 때 마다, 기존 출력물들은 제거해주는 모듈
from IPython.display import clear_output

In [ ]:
# 모델 모듈
import xgboost
from xgboost import XGBRegressor, plot_importance
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, LinearRegression, ElasticNet
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
stats+=['HIT_RATE']

In [ ]:
# 누적 및 lag stat features 이름
stats_cum = ['cum_' + i for i in stats]
stats_16 = ['16_' + i for i in stats]
stats_17 = ['17_' +i for i in stats]
stats_18 = ['18_' +i for i in stats]

In [ ]:
X_cols=stats_cum+stats_16+stats_17+stats_18
#X_cols=stats_16+stats_17+stats_18
Y_col=['19_HIT_RATE'] #19_AB, 19_HIT

In [ ]:
#### 학습시킬 모델
model_dict = {'xgb':XGBRegressor(n_estimators=110, learning_rate=0.05 ,max_depth=3, min_samples_split=400, random_state=23),
    'lgbm':LGBMRegressor(),
    'rf':RandomForestRegressor(),
    'svr':SVR(),
    'knn':KNeighborsRegressor(),
    'reg':LinearRegression(),
    'ridge':Ridge(),
    'lasso':Lasso()}

In [ ]:
train=train18
test=test18

In [ ]:
X_train = train[X_cols]; Y_train = train[Y_col]
X_test = test[X_cols]; Y_test = test[Y_col]
diff=[]

#모델 여러개 돌려보기
model_list=['xgb','lgbm','rf','svr','reg','ridge','lasso']
for m in model_list:
    model = model_dict[m]
    if m=='rf': Y_train=Y_train.values.ravel()  #rf에서의 오류 없애기 위함
    weight = train['mean']
    model.fit(X_train, Y_train, sample_weight=weight)
    Y_predict = model.predict(X_test)

    r2=r2_score(Y_test, Y_predict)

    a=Y_test.assign(predict=Y_predict)
    a['diff']=a['HIT_RATE']-a['predict']
    print(m,"\t>r2 :", r2,'\ttrain점수 :{:.3f}'.format(model.score(X_train,Y_train)),"\t차이합 :",abs(a['diff']).sum())
    diff.append(r2)

In [ ]:
#r2가 가장 큰 모델 돌리기
model = model_dict[model_list[diff.index(max(diff))]]
if m=='rf': Y_train=Y_train.values.ravel()  #rf에서의 오류 없애기 위함
weight = train['mean']
model.fit(X_train, Y_train, sample_weight=weight)
Y_predict = model.predict(X_test)
print(model_list[diff.index(min(diff))])

### 예측값과 실제값 차이

In [ ]:
# 예측값과 실제값의 시각화
plt.scatter(Y_test, Y_predict)
plt.xlabel("실제 값")
plt.ylabel("예측 값")

In [ ]:
#예측값과 실제값 차이
a=Y_test
a=a.assign(predict=Y_predict)
a['diff']=a['HIT_RATE']-a['predict']
a

In [ ]:
#차이 합
abs(a['diff']).sum()